In [1]:
import re

class CAZy_data:
    def __init__(self,filename1,filename2):
        self.data,self.acc,self.seq=[],[],[]
        with open(filename1,'r',encoding='utf-8') as inpt:
            for each in inpt:
                self.data.append(each.rstrip().split('$'))
                
        with open(filename2,'r',encoding='utf-8') as inpt1:
            for each1 in inpt1:
                if each1.startswith('>'):
                    self.acc.append(each1.rstrip())
                else:
                    self.seq.append(each1.rstrip())
            
                  
    def data_fetch(self,typ,position):
        typ_data=[]
        if typ=='all':
            typ_data=self.data
        else:
            for each in self.data:
                mult=each[position].split(' ')
                if len(mult)==1:#### In case typ = EC, Multi EC number and protein with no EC number are ignore.
                    if mult[0]==typ:
                        typ_data.append(each)
        return typ_data
    
    def EC_GH(self,ec_no,gh_fam):
        self.fasta=[]
        cazy_ec=self.data_fetch(ec_no,1)
        cazy_gh=self.data_fetch(gh_fam,-1)
        self.common_data=[[i[0],i[1],i[3],i[4],i[-2],i[-1]] for i in cazy_ec if i in cazy_gh]
        rm_prt, rm_prt_fasta=[],[]
        for each in range(len(self.common_data)):
            t=self.common_data[each]
            if self.prtn_filter(t[0]):
                all_acc=t[3].split(' ')
                if all_acc[0]!='':
                    for e_acc in all_acc:
                        e_seq=self.seq_fetch(e_acc)
                        try:
                            create_error=0/len(e_seq) # to remove accession number which doesnt have hits
                            self.fasta.append(f'>{e_acc}${t[0]}${t[1]}${t[2]}${t[-2]}${t[-1]}')
                            self.fasta.append(e_seq[0])
                        except ZeroDivisionError:
                            rm_prt_fasta.append(e_acc)
            else:
                rm_prt.append(t)
#         print('Total number of sequences:',len(self.fasta)/2)
#         print('Number of removed partial or fragment proteins (CAZy):',len(rm_prt))
#         print('Number of removed partial or fragment proteins (Fasta):',len(rm_prt_fasta))
        return self.fasta,rm_prt,rm_prt_fasta
    def prtn_filter(self,prt_name):
        hit=1
        if re.search('partial|fragment',prt_name.lower()):
            hit-=1
        return hit
            
    def seq_fetch(self,accession):
        hits=[]
        temp=0
        for each in range(len(self.acc)):
            if re.search(f'{accession}\D',self.acc[each]):
                temp+=1
                
                if self.prtn_filter(self.acc[each]): # remove partial| fragment accession numbers from GenBank description
                    hits.append(self.seq[each])
        if temp>1:
            print(f'Multiple hits for {accession}')
        elif temp==0:
            print(f'No hits for {accession}')
        return hits
     
In_data=CAZy_data('cazy_char_10_6_22.txt','char_gh_23_6_22.txt')


In [31]:
cazy_acc_seq,rm_cazy,rm_genbank=In_data.EC_GH('3.2.1.4','GH45') # write all to fetch all the EC number or all the GH family

No hits for XP_324478.1


In [32]:
import pandas as pd
import os

In [33]:
#splitting sequence and accession IDs
acc,seq=[],[]
for ii in cazy_acc_seq:
    if ii.startswith('>'):
        acc.append(ii.rstrip())
    else:
        seq.append(ii.rstrip()) 

In [20]:
#creating a dictionary with only the reqiuired information with the accession ID
acc_seq={}
for i in range(len(acc)):
    accc=acc[i].split('$')
    acc_no=accc[0]
    x='>'+acc_no[1:]+'&'+accc[2]+'&'+accc[4]+'&'+accc[5]
    acc_seq[x] = seq[i]


In [21]:
#the file with the required info
y=len(seq)
with open('all_GH5.fasta','w',encoding='utf-8') as otp:
    for i in acc_seq.keys():
        seq=acc_seq.get(i)
        otp.write(i+'\n'+seq+'\n')
    

In [34]:
acc

['>ACE82688.1$endo-β-1,4-glucanase B / 45A (CelB;EGB;CJA_0374)  (Cel45A)$3.2.1.4$Cellvibrio japonicus Ueda107$Bacteria$GH45',
 '>CAA36844.1$endo-β-1,4-glucanase B / 45A (CelB;EGB;CJA_0374)  (Cel45A)$3.2.1.4$Cellvibrio japonicus Ueda107$Bacteria$GH45',
 '>ACX75523.1$endo-β-1,4-glucanase (F45;Fisuc_1933;FSU_2443)$3.2.1.4$Fibrobacter succinogenes subsp. succinogenes S85$Bacteria$GH45',
 '>ADL26053.1$endo-β-1,4-glucanase (F45;Fisuc_1933;FSU_2443)$3.2.1.4$Fibrobacter succinogenes subsp. succinogenes S85$Bacteria$GH45',
 '>AAP49340.1$endoglucanase B (GnuB)$3.2.1.4$uncultured bacterium$Bacteria$GH45',
 '>AKC91346.1$cellulase (Eng-2;ArENG2)$3.2.1.4$Adineta ricciae$Eukaryota$GH45',
 '>ABR92637.1$endo-β-1,4-glucanase (EG27I)$3.2.1.4$Ampullaria crossean$Eukaryota$GH45',
 '>ABR92638.1$endo-β-1,4-glucanase (EG27II)$3.2.1.4$Ampullaria crossean$Eukaryota$GH45',
 '>AFN89565.1$cellulase 45A (AmCel-45A)$3.2.1.4$Anoplophora chinensis$Eukaryota$GH45',
 '>BAP19116.1$cellulase 21K  (AkEG21;AkEG22)$3.2.1.4$A